<h1> Procesamiento de las imágenes </h1>



Para el procesamiento de las imagenes, el primer paso es leerlas de la carpeta correspondiente, la cual se encuentra en la carpeta srcImg, por lo tanto:

# Librerias en general

In [3]:
"""
Biblioteca:
-   cv2             Manejo de imágenes
-   numpy           Manejo de arreglos de pixeles
(Prueba)-   cv2_imshow      Parche para mostrar imágenes en Colab

-   time            Biblioteca para debuggear
-   matplotlib      Manejo de las impresiones de imagenes
"""
%matplotlib inline

import os
import cv2
import numpy as np
import random
#from google.colab.patches import cv2_imshow

import time 
from matplotlib import pyplot as plt

# Dirección principal
dir = "c:/Users/jose_/Desktop/Tesis/Programa"
# Direccion de guardado de imágenes
outDir = os.getcwd()+"\Imagenes"
# Direccion de carga de videos
inDir = os.getcwd()+"\Videos"
#Dirección de guardado de cada canal
outChan = os.getcwd()+"\Canales"
#Dirección de guardado de los histogramas de cada canal
outHist = os.getcwd()+"\Histogramas"



# Obteniendo las imagenes
global imgs 

imgs = list()

In [20]:
"""
En esta sección se añadirán las funciones realizadas, revisadas y aprobadas para la realización del filtro
"""

def getImages(dir):
    """
        getImages nos ayuda a cargar las imagenes de la carpeta Imagenes y guardarlas en la lista imgs, ademas 
        de guardar el nombre de cada imagen en la misma lista, es decir, guardará las imagenes como una tupla (Imagen, Nombre)  

        :param dir: Dirección local de la carpeta Imagenes
        :type dir: String
        :return: Lista de tuplas (Imagen, Nombre)
        :rtype: List
    """

    # Obtenemos el nombre de todos los archivos que se encuentran en la carpeta Imagenes
    files = os.listdir(dir)                 

    # Creamos un bucle para cada elemento dentro de la carpeta    
    for file in files:
        # Creamos la variable de la direccion de cada imagen
        image_path = dir + "/" + file           
       
        # Leemos la imagen
        image = cv2.imread(image_path)   
       
        # Si por alguna cuestion no se puede leer la imagen o el archivo no es una imagen se descarta en automatico
        if image is None:                          
            continue
        
        # Agregamos la tupla de la imagen y su nombre a la lista imgs
        imgs.append((image, file))

# -----------------------------------------------------------------------------------------------------------------------------------------------

def selectContours():
    """
        Método para obtener los limites de los bordes de la pecera, con el fin de recortar las imagenes del mismo tamaño

        :return: min, max -> Representan los bordes izquierdos y derechos
        :rtype: int, int
    """

    # Valor minimo para la linea extrema izquierda, en este caso se puso un valor alto para que se actualice al encontrar las lineas
    min = 0
    # Valor maximo para la linea extrema derecha, en este caso se puso un valor alto para que se actualice al encontrar las lineas
    max = imgs[0][0].shape[1]


    for i, _ in imgs:
        img = i.copy()

        # Cambio de imagen en color a imagen a escala de grises 
        imagen_grises = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        ret, _ = cv2.threshold(imagen_grises, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU )

        # Canny es el detector de bordes
        # Imagen_ grises        Es la imagen fuente a utilizar en escala de grises
        # 50                    Valor de umbral minimo, a partir del cual se considera algo un borde
        # 150                   Valor de umbral maximo, a partir del cual se considera algo un borde
        # ApertureSize          El ancho de la linea de bordes que se deja en la imagen
        bordes = cv2.Canny(imagen_grises, 0, ret)

        # Detector de lineas
        # Bordes es la imagen resultante de aplicar Canny 
        # 1 es el radio de resolución, es decir, los saltos entre pixeles
        # pi/180 Angulo de busqueda, normalmente lleva el valor de 1 grado pero representado en coordenadas polares
        # 300 Threshld, pero no es un valor de umbral, se refiere mas bien a la cantidad de puntos minimos para considerar algo una linea 
        lineas = cv2.HoughLines(bordes, 1, np.pi/180, 200)

        # Recorremos las lineas encontradas buscando las de nuestro interés
        for linea in lineas:
            # Obtenemos los valores del radio y angulo del punto que representa una linea
            rho, theta = linea[0]
            
            # Iniciamos la conversión de coordenadas polares a cartesianas
            a = np.cos(theta)
            b = np.sin(theta)

            # Obtenemos los valores x0 y y0
            x0 = a * rho

            # A partir de los valores de x0 y y0, creamos dos puntos ((x1,y1), (x2,y2)) con una constante de 1500 para trazar la linea
            x1 = int(x0 + 100 * (-b))
            x2 = int (x0 - 100 * (-b))

            if x1 == x2:
                if x1 <= i.shape[1]/2 and x1 > min:
                    min = x1
                if x1 >= i.shape[1]/2 and x1 < max:
                    max = x1

    return min, max

# -----------------------------------------------------------------------------------------------------------------------------------------------

def cropImages(min, max):
    """
        Método para recortar las imagenes

        :param min: Limite izquierdo para recortar la imagen
        :type min: int
        :param max: Limite derecho para recortar la imagen
        :type max: int
    """
    crop = list()
    for img, name in imgs:
        # cv2.rectangle(img, (min, - img.shape[0]) , (max, img.shape[0]), (255,0,0), 10)
        crop.append(img[:, min: max])
    
    for i in range(0, len(imgs)-1):
        imgs[i] = (crop[i], imgs[i][1])
    
# -----------------------------------------------------------------------------------------------------------------------------------------------

def showImages():
    """
        Método para mostrar las imagenes
    """
    for img, name in imgs:
        cv2.imshow(name, cv2.resize(img, (512,512)))
        cv2.waitKey(400)
        cv2.destroyAllWindows()


def componenteConexa(img, areaLim):
    contours, _ = cv2.findContours(img,  
                cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 

    # print("He encontrado {} objetos".format(len(contours)))

    for cnt in contours:
    #         for i in range(0, len(cnt)):
    #                 cv2.drawContours(img, cnt, i, (0, 255, 0), 3)
    #                 cv2.imshow('Contours', cv2.resize(img, (512,512))) 
            area =  cv2.contourArea(cnt)

            if(area > areaLim ):
                print(area)
                cv2.drawContours(img, cnt, -1, (0, 255, 0), 3) 

    cv2.imshow('Contours', cv2.resize(img, (512,512))) 
    cv2.waitKey(400)
    cv2.destroyAllWindows()

In [5]:
# Aqui mandamos a llamar el metodo getImages, este va aparte ya que no siempre hay que leer las imagenes
getImages(outDir)

In [6]:
min, max = selectContours()
cropImages(min, max)

In [7]:

showImages()

## Procesamiento de las imágenes

Para el procesamiento de las imágenes se probaran distintos métodos.

-------------------------------------------------------------------------------------------------------

### Los métodos a probar son:
1. Substracción [cada T frames]

2. OTSU + Componente conexa

3. GAUSS + OTSU + Componente conexa

4. OTSU + SUBSTRACCION + Componente conexa

5. GAUSS + ADAPTATIVO +  Componente conexa

6. PIXELLIB

----------------------------------------------------------------------------------------------------


# <h1> 1.  Substracción </h1>

In [8]:
# Primera prueba Substracción de fotos

# Usamos el timelapse que se utilizo para la campura de los fotogramas
timelapse = 1
# Lo pasamos a numero entero el incremento
rate = int(timelapse * 10)

# Creamos una lista de imagenes donde se guardaran los resultados
images = list()

# Creamos un bucle para recorrer las imagenes y substraer la imagen i a la i + rate creando un bucle en el arreglo, es decir
# si nos llegaramos a pasar del index del array, este hara el modulo e iniciará de nuevo, por ejemplo 125 % 124 = 1
for i in range(0, len(imgs)-1):
    images.append(cv2.subtract(imgs[i][0], imgs[(i+rate)%(len(imgs)-1)][0]))
    cv2.imshow("Image", cv2.resize(images[i], (512,512)))
    cv2.waitKey(200)
cv2.destroyAllWindows()

# <h1> 2.  Substracción + Threshold </h1>

In [10]:
for img in images:
     
    imagen_grises = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imagen_grises, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU )

    cv2.imshow("Image", cv2.resize(thresh, (512,512)))
    cv2.waitKey(300)
cv2.destroyAllWindows()

# <h1> 3. OTSU + Componente conexa </h1>

In [13]:
for i in range(0, len(imgs)-1):
        img = imgs[i][0].copy()

        # Cambio de imagen en color a imagen a escala de grises 
        imagen_grises = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        ret, _ = cv2.threshold(imagen_grises, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU )
        # ret, _ = cv2.threshold(imagen_grises, 0, 255, cv2.THRESH_OTSU )

        # Canny es el detector de bordes
        # Imagen_ grises        Es la imagen fuente a utilizar en escala de grises
        # 0                     Valor de umbral minimo, a partir del cual se considera algo un borde
        # ret                   Valor de umbral maximo, a partir del cual se considera algo un borde
        # ApertureSize          El ancho de la linea de bordes que se deja en la imagen
        bordes = cv2.Canny(imagen_grises, 0, ret)

        cv2.imshow("Image", cv2.resize(bordes, (512,512)))
        cv2.waitKey(200)
cv2.destroyAllWindows()


# <h1> 4. GAUSS + OTSU + Componente conexa </h1>

In [24]:
gausImages = list()

for i in range(0, len(imgs)-1):
# for i in range(0, 10):
        img = imgs[i][0].copy()

        # Cambio de imagen en color a imagen a escala de grises 
        imagen_grises = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        dst = cv2.GaussianBlur(imagen_grises, (5,5), cv2.BORDER_DEFAULT)

        ret, _ = cv2.threshold(dst, 0, 255, cv2.THRESH_OTSU )

        # Canny es el detector de bordes
        # Imagen_ grises        Es la imagen fuente a utilizar en escala de grises
        # 50                    Valor de umbral minimo, a partir del cual se considera algo un borde
        # 150                   Valor de umbral maximo, a partir del cual se considera algo un borde
        # ApertureSize          El ancho de la linea de bordes que se deja en la imagen
        # bordes = cv2.Canny(dst, 0, ret)
        bordes = cv2.Canny(dst, 1, ret)

        gausImages.append(bordes)
        # componenteConexa(bordes, 0)

        cv2.imshow("Image", cv2.resize(bordes, (512,512)))
        cv2.waitKey(500)
cv2.destroyAllWindows()

# <h1> 4.2 GAUSS + OTSU + Componente Conexa + substracción </h1>

In [26]:
for i in range(0, len(gausImages)-1):
    img = cv2.subtract(gausImages[i], gausImages[(i+rate)%(len(imgs)-1)])
    cv2.imshow("Image", cv2.resize(img, (512,512)))
    cv2.waitKey(200)
cv2.destroyAllWindows()

# <h1> 5. OTSU + SUBSTRACCION + Componente conexa </h1>

In [28]:
imagesOtsu = list()
rate = timelapse*10

for i in range(0, len(imgs)-1):
        img = imgs[i][0].copy()

        # Cambio de imagen en color a imagen a escala de grises 
        imagen_grises = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        ret, tresh = cv2.threshold(imagen_grises, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU )

        imagesOtsu.append(thresh)
        # ret, _ = cv2.threshold(imagen_grises, 0, 255, cv2.THRESH_OTSU )


for i in range(0, len(imagesOtsu)-1):

        images.append(cv2.subtract(imagesOtsu[i], imagesOtsu[(i+rate)%(len(imgs)-1)]))
        # Canny es el detector de bordes
        # Imagen_ grises        Es la imagen fuente a utilizar en escala de grises
        # 0                     Valor de umbral minimo, a partir del cual se considera algo un borde
        # ret                   Valor de umbral maximo, a partir del cual se considera algo un borde
        # ApertureSize          El ancho de la linea de bordes que se deja en la imagen
        bordes = cv2.Canny(images[i], 0, ret)
        
        cv2.imshow("Image", cv2.resize(bordes, (512,512)))
        cv2.waitKey(400)

cv2.destroyAllWindows()

# <h1> 6. GAUSS + ADAPTATIVO +  Componente conexa </h1>

In [30]:

for i in range(0, len(imgs)-1):
    img = imgs[i][0].copy()

    imagen_grises = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    dst = cv2.GaussianBlur(imagen_grises, (11,11), cv2.BORDER_DEFAULT)

    th2 = cv2.adaptiveThreshold(dst, 255, cv2.ADAPTIVE_THRESH_MEAN_C,\
                                cv2.THRESH_BINARY,11,2)
    th3 = cv2.adaptiveThreshold(dst, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                                cv2.THRESH_BINARY,11,2)
    
    cv2.imshow("Thresh MEAN C", cv2.resize(th2, (512,512)))
    cv2.imshow("Thresh GAUSSIAN C", cv2.resize(th3, (512,512)))

    cv2.waitKey(400)
cv2.destroyAllWindows()

# <h1> 6. PIXELLIB </h1>

No lo implemente en maquina fisica, solo en Colab.

<href> https://colab.research.google.com/drive/1K7l5QOJADiMV02Er_aRMxuBJGeL-0HVr?usp=sharing </href>